# Sparse coding and dictionary learning on Graph

# sparse coding

## objective

- minimize reconstruction error of recreating signal $\mathbf{y} \in \mathbb{R}^{n}$ as a linear combination of atoms in dict $\mathcal{D} \in \mathbb{R}^{n \times K}$ with sparse coefficient $\mathbf{x} \in \mathbb{R}^{K}$

$$
\underset{\mathbf{x} }{\arg\min} \left\|\mathbf{x}\right\|_0\text{ subject to } \left\| \mathbf{y} - \mathcal{D} \mathbf{x} \right\|_2^2  \leq \epsilon 
$$

- $\mathbf{y} \in \mathbb{R}^n$ is an input graph signal

- $\mathcal{D}=\left\{\mathbf{d}_1, ..., \mathbf{d}_p\right\}$ is a overcomplete dictionary of atoms (column vector).

    overcomplete means atoms don't have to be orthonormal or be eigen basis.

    atoms $\mathbf{d}$ can be chosen or learned, e.g., eigenvectors of graph laplacian, graph wavelets, subdicts of polynomials of graph laplacian

## solving

- this is a non-convex optimization due to $l_0$ norm, which is the number of non-zero entries in sparse coefficient $\mathbf{x} \in \mathbb{R}^p$

- classic sparse coding is solved by **matching pursuit**. 

    1. Find the atom that loads the highest to the signal

    2. Compute a  “residual” by using an orthogonal projection to the atoms selected so far

    Repeats step 1, Proceeds until convergence

# dictionary learning

k-SVD: best-known dict learning algorithm, learn a dictionary $\mathcal{D}$ and sparse coding $X$ simultaneously

https://legacy.sites.fas.harvard.edu/~cs278/papers/ksvd.pdf

- objective: find best dictionary $\mathcal{D} \in \mathbb{R}^{n \times N}$ for sparse represenation of N samples $Y \in \mathbb{R}^{n \times p} = \left\{\mathbf{y_i}\right\}_{i=1}^N$ with sparse coefficient $X \in \mathbb{R}^{K \times N}$, $\mathbf{x_i} \in \mathbb{R}^{K}$ is column vector of $X$

$$
\underset{\mathcal{D}, X}{\arg\min} \left\| Y - \mathcal{D} X\right\|_F^2 \text{ subject to } \forall i =1,2,..., N \left\|\mathbf{x_i}\right\|_0 \leq T
$$

## algorithm

initialization: set dictionary matrix $\mathcal{D}^{(0)} \in \mathbb{R}^{n \times N}$ with $l_2$ normalized columns. set $J=1$

repeat untill convegence (stopping rule)

**sparse coding stage**: use any pursuit algorithm to compute representation vector $\mathbf{x_i} \in \mathbb{R}^{K}$ for each sample $\mathbf{y_i} \in \mathbb{R}^{n}$ 

$$
i = 1,2..., N, \underset{\mathbf{x_i}}{\arg\min} \left\| \mathbf{y_i} - \mathcal{D} \mathbf{x_i}\right\|_2^2 \text{ subject to } \left\|\mathbf{x_i}\right\|_0 \leq T
$$

**codebook update stage**: for each column $\mathbf{d}_k$ in $\mathcal{D}^{(J-1)} \in \mathbb{R}^{n \times N}$ ($k=1,2,..., K$), update it by

- define group of samples that use this atom $\phi_k$

$$
\omega_k =\left\{i|1\leq i \leq N, [\mathbf{x_i}]_k \neq 0\right\}
$$

- compute overall representation error matrix $E_k$

$$
E_k = Y - \sum_{j \neq k}^K \mathbf{d}_j \mathbf{X}_{[j, :]}
$$

- restrict $E_k$ by choosing only columns corresponding to $\omega_k$ and obtain $E_k^R$

- SVD decompose restricted error matrix $E_k^R = USV^T$

    update dictionary column $\tilde{\mathbf{d}}_k$ to be first column of $U$, update coefficient vector $\mathbf{X}_{[k, :]}$ to be first column of $V$ multipled by 2 x 2 diagonal matrix with entries 1

- set $J=J+1$

## key step: error when an atom is removed

$$ 
\begin{align}
\left\| Y - \mathcal{D} X\right\|_F^2 
&=  \left\| Y - \sum_{j =1}^K \mathbf{d}_j \mathbf{X}_{[j, :]}\right\|_F^2 \\[1em]
&=  \left\| \left(Y - \sum_{j \neq k}^K \mathbf{d}_j \mathbf{X}_{[j, :]} \right) -  \mathbf{d}_k \mathbf{X}_{[k, :]}\right\|_F^2 \\[1em]
&= \left\| E_k -  \mathbf{d}_k \mathbf{X}_{[k, :]}\right\|_F^2 \\[1em]
\end{align}
$$

# polynomial dictionary

- objective: learn a dictionary $\mathcal{D}$ as concatenation of subdictionaries $[\mathcal{D}_1, ..., \mathcal{D}_S]$

$$
\underset{\alpha}{\arg\min} \left\| Y - \mathcal{D} X\right\|_F^2 +\mu \left\|\alpha \right\|_2^2\\[1em]
\text{subject to } \mathcal{D}_s = \sum_{k=0}^K \alpha_{sk} L^k\ \ \forall s \in \left\{1, 2..., S\right\}
$$

where vector $\alpha \in \mathbb{R}^{(K+1)S}$ is coefficient

$L$ is Graph Laplacian

constraint: each subdictionary (atom) $\mathcal{D}_s$ is polynomial of graph laplacian

# graph denoising coding

- objective: convex, cen be solved faster by SGD

$$
\underset{X}{\arg\min} \left\| Y - \mathcal{D} X\right\|_F^2 + \gamma (DX)^TL(DX)
$$

$(DX)^TL(DX)$: a quadratic form constrains signal to be smooth instead of sparse

$\gamma$: regularization parameter